# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2022 12:00AM,239367,10,Enova-10117,Emerginnova,Released
1,Jun 13 2022 12:00AM,239367,10,Enova-10118,Emerginnova,Released
2,Jun 10 2022 6:05PM,239383,16,EMERSA-8128022,"Emersa Waterbox, LLC",Released
3,Jun 10 2022 4:39PM,239375,21,617088,"NBTY Global, Inc.",Released
4,Jun 10 2022 4:22PM,239373,16,8127497,Sartorius Stedim Filters Inc.,Released
5,Jun 10 2022 4:13PM,239372,16,8127494,Sartorius Stedim Filters Inc.,Released
6,Jun 10 2022 2:11PM,239355,20,SSNA-2173662705,Sartorius Stedim North America,Released
7,Jun 10 2022 2:04PM,239353,20,ALUR038152223,Alumier Labs Inc.,Released
8,Jun 10 2022 2:04PM,239353,20,ALUR074904655,Alumier Labs Inc.,Executing
9,Jun 10 2022 2:04PM,239353,20,ALUR119861207,Alumier Labs Inc.,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,239367,Released,2
13,239372,Released,1
14,239373,Released,1
15,239375,Released,1
16,239383,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239367,NaN,NaN,2.0
239372,NaN,NaN,1.0
239373,NaN,NaN,1.0
239375,NaN,NaN,1.0
239383,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239318,0.0,0.0,11.0
239319,0.0,0.0,9.0
239323,5.0,3.0,1.0
239332,0.0,0.0,4.0
239340,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239318,0,0,11
239319,0,0,9
239323,5,3,1
239332,0,0,4
239340,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239318,0,0,11
1,239319,0,0,9
2,239323,5,3,1
3,239332,0,0,4
4,239340,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239318,,,11
1,239319,,,9
2,239323,5,3,1
3,239332,,,4
4,239340,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2022 12:00AM,239367,10,Emerginnova
2,Jun 10 2022 6:05PM,239383,16,"Emersa Waterbox, LLC"
3,Jun 10 2022 4:39PM,239375,21,"NBTY Global, Inc."
4,Jun 10 2022 4:22PM,239373,16,Sartorius Stedim Filters Inc.
5,Jun 10 2022 4:13PM,239372,16,Sartorius Stedim Filters Inc.
6,Jun 10 2022 2:11PM,239355,20,Sartorius Stedim North America
7,Jun 10 2022 2:04PM,239353,20,Alumier Labs Inc.
23,Jun 10 2022 1:41PM,239347,10,"Methapharm, Inc."
24,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc."
25,Jun 10 2022 12:12PM,239332,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2022 12:00AM,239367,10,Emerginnova,,,2
1,Jun 10 2022 6:05PM,239383,16,"Emersa Waterbox, LLC",,,1
2,Jun 10 2022 4:39PM,239375,21,"NBTY Global, Inc.",,,1
3,Jun 10 2022 4:22PM,239373,16,Sartorius Stedim Filters Inc.,,,1
4,Jun 10 2022 4:13PM,239372,16,Sartorius Stedim Filters Inc.,,,1
5,Jun 10 2022 2:11PM,239355,20,Sartorius Stedim North America,,,1
6,Jun 10 2022 2:04PM,239353,20,Alumier Labs Inc.,2,7,7
7,Jun 10 2022 1:41PM,239347,10,"Methapharm, Inc.",,,1
8,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc.",,,1
9,Jun 10 2022 12:12PM,239332,10,Emerginnova,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 12:00AM,239367,10,Emerginnova,2,,
1,Jun 10 2022 6:05PM,239383,16,"Emersa Waterbox, LLC",1,,
2,Jun 10 2022 4:39PM,239375,21,"NBTY Global, Inc.",1,,
3,Jun 10 2022 4:22PM,239373,16,Sartorius Stedim Filters Inc.,1,,
4,Jun 10 2022 4:13PM,239372,16,Sartorius Stedim Filters Inc.,1,,
5,Jun 10 2022 2:11PM,239355,20,Sartorius Stedim North America,1,,
6,Jun 10 2022 2:04PM,239353,20,Alumier Labs Inc.,7,7,2
7,Jun 10 2022 1:41PM,239347,10,"Methapharm, Inc.",1,,
8,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc.",1,,
9,Jun 10 2022 12:12PM,239332,10,Emerginnova,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 12:00AM,239367,10,Emerginnova,2,,
1,Jun 10 2022 6:05PM,239383,16,"Emersa Waterbox, LLC",1,,
2,Jun 10 2022 4:39PM,239375,21,"NBTY Global, Inc.",1,,
3,Jun 10 2022 4:22PM,239373,16,Sartorius Stedim Filters Inc.,1,,
4,Jun 10 2022 4:13PM,239372,16,Sartorius Stedim Filters Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 12:00AM,239367,10,Emerginnova,2,NaN,NaN
1,Jun 10 2022 6:05PM,239383,16,"Emersa Waterbox, LLC",1,NaN,NaN
2,Jun 10 2022 4:39PM,239375,21,"NBTY Global, Inc.",1,NaN,NaN
3,Jun 10 2022 4:22PM,239373,16,Sartorius Stedim Filters Inc.,1,NaN,NaN
4,Jun 10 2022 4:13PM,239372,16,Sartorius Stedim Filters Inc.,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 12:00AM,239367,10,Emerginnova,2,0.0,0.0
1,Jun 10 2022 6:05PM,239383,16,"Emersa Waterbox, LLC",1,0.0,0.0
2,Jun 10 2022 4:39PM,239375,21,"NBTY Global, Inc.",1,0.0,0.0
3,Jun 10 2022 4:22PM,239373,16,Sartorius Stedim Filters Inc.,1,0.0,0.0
4,Jun 10 2022 4:13PM,239372,16,Sartorius Stedim Filters Inc.,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,957386,8,0.0,0.0
16,1196765,23,0.0,0.0
20,718031,9,10.0,7.0
21,239375,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,957386,8,0.0,0.0
1,16,1196765,23,0.0,0.0
2,20,718031,9,10.0,7.0
3,21,239375,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,8,0.0,0.0
1,16,23,0.0,0.0
2,20,9,10.0,7.0
3,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,16,Released,23.0
2,20,Released,9.0
3,21,Released,1.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,20,21
Status,,,,
Completed,0.0,0.0,7.0,0.0
Executing,0.0,0.0,10.0,0.0
Released,8.0,23.0,9.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,20,21
0,Completed,0.0,0.0,7.0,0.0
1,Executing,0.0,0.0,10.0,0.0
2,Released,8.0,23.0,9.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,20,21
0,Completed,0.0,0.0,7.0,0.0
1,Executing,0.0,0.0,10.0,0.0
2,Released,8.0,23.0,9.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()